In [36]:
import pandas as pd
import requests


In [56]:

def ListarOpcoes(ativoObj, vencimento):    
    url = f'https://opcoes.net.br/listaopcoes/completa?cache={vencimento}_16h1&au=False&uinhc=0&idLista=ML&idAcao={ativoObj}&listarVencimentos=true&cotacoes=true'
    r = requests.get(url).json()
    l = [[ativoObj, vencimento, i[0].split('_')[0], i[2], i[3], i[5], i[8]] for i in r['data']['cotacoesOpcoes']]
    return pd.DataFrame(l, columns= ['ativoObj', 'vencimento', 'ativo', 'tipo', 'modelo', 'strike', 'preco'])

In [57]:
def ListarTudo(ativoObj):
    url = f'https://opcoes.net.br/listaopcoes/completa?cache=2023-5-22_16h1&au=False&uinhc=0&idLista=ML&idAcao={ativoObj}&listarVencimentos=true&cotacoes=true'
    r = requests.get(url).json()
    vencimentos = [i ['value'] for i in r ['data']['vencimentos']]
    df = pd.concat([ListarOpcoes(ativoObj, vencimento) for vencimento in vencimentos])
    return df

In [61]:
ListarTudo("PETR4")

,ativoObj,vencimento,ativo,tipo,modelo,strike,preco
0,PETR4,2023-06-16,PETRF1,CALL,A,12.97,13.00
1,PETR4,2023-06-16,PETRF12,CALL,E,12.22,13.60
2,PETR4,2023-06-16,PETRF32,CALL,E,9.72,16.03
3,PETR4,2023-06-16,PETRF39,CALL,A,19.97,6.11
4,PETR4,2023-06-16,PETRF64,CALL,E,12.72,13.65
...,...,...,...,...,...,...,...
233,PETR4,2025-05-16,PETRR500,PUT,E,25.97,0.91
234,PETR4,2025-05-16,PETRR502,PUT,E,50.22,23.79
235,PETR4,2025-05-16,PETRR503,PUT,E,30.97,4.81
236,PETR4,2025-05-16,PETRR513,PUT,E,31.97,5.71
